In [1]:
import pandas as pd
import numpy as np
import numpy.ma as ma 
import random 

tsp29 = pd.read_excel("TSP_29.xlsx", index_col=[0])
tsp48 = pd.read_excel("Dane_TSP_48.xlsx", index_col=[0])
tsp76 = pd.read_excel("Dane_TSP_76.xlsx", index_col=[0])
tsp127 = pd.read_excel("Dane_TSP_127.xlsx", index_col=[0])

def df_to_array(data):
    cities_lenght = np.array(data)
    city_number = list(range(0,len(data)))
    return (cities_lenght,city_number)



In [2]:
def randomTour(cities_lenght,city_number):
    tour = []
    city_number_random = city_number.copy()
    for i in range(len(city_number)):
        random_step = city_number_random[random.randint(0,len(city_number_random)-1)]
        tour.append(random_step)
        city_number_random.remove(random_step)
    return tour

In [3]:
def tourDisatance(cities_lenght,city_number,tour): 
    tour_distance = 0
    for i in range(len(city_number)):
        tour_distance = tour_distance + cities_lenght[tour[i-1]][tour[i]] #pierwsze doda ostatnie miasto z pierwszym a później sprawdzi po koleji
    return tour_distance


In [4]:
def randomTour_max(cities_lenght,city_number,max_distance):
    tour = []
    city_number_random = city_number.copy()
    for i in range(len(city_number)):
        random_step = city_number_random[random.randint(0,len(city_number_random)-1)]
        tour.append(random_step)
        city_number_random.remove(random_step)
    if (tourDisatance(cities_lenght,city_number,tour)>max_distance):
        tour = randomTour_max(cities_lenght,city_number,max_distance)
    return tour

In [5]:
for i in range(100):
    cities_lenght,city_number = df_to_array(tsp127)
    x = randomTour_max(cities_lenght,city_number,580000)
    print(tourDisatance(cities_lenght,city_number,x))


579530.3795557554
579970.0728548001
574954.5307078317
571318.7617654951
578780.5611466999
571701.6283677064
568193.7850417878
573474.1294816596
566051.9551119644
576272.6787702782
566483.6028288128
571471.6770939407
577741.1811673808
572326.4257198104
560652.5578806312
576622.5197018597
571051.2589026716
576885.4449490197
577491.9108009782
565427.0812232106
566142.9106483446
579395.9490562618
579431.6570282975
567229.0989433723
577115.5731697007
569250.6792078242
575393.7843129338
571890.164292777
570117.1947582308
579518.4023862589
576257.773940799
574100.6721644057
575689.974105833
577709.7186122906
575161.6450422131
574654.296215508
573917.0264921408
579286.8627834915
579538.7750019574
575286.5890027682
578329.8567467058
570370.1294389251
579270.0030908522
578205.8188832143
579292.124108323
575082.6929701741
575479.7144217748
560837.5747725829
578572.6725277404
575682.4752568675
579756.6650431656
577309.1703228928
566158.5618424327
573566.4017764502
578246.371583873
569340.647739109

In [6]:
def Neighbours(tour,city_number):
    neighbours = []
    for i in range(0,len(city_number)):
        for j in range(i+1,len(city_number)):
            neighbour = tour.copy()
            neighbour[i] =  tour[j]
            neighbour[j] = tour[i] 
            neighbours.append(neighbour) #lista z wszystkimi sąsiadami od podanego _tour_. Sąsiedztwo typ [1,2,3] -> [2,1,3],[3,2,1][1,3,2](wymiani kązdy zkażdym)
    return neighbours


In [7]:
def Neighbours2(tour,city_number): 
    neighbours = []
    for i in range(0,len(city_number)):
        for j in range(0,len(city_number)):
            neighbour = tour.copy()
            step = neighbour.pop(i)
            neighbour.insert(j,step)
            neighbours.append(neighbour) #lista z wszystkimi sąsiadami od podanego _tour_. Sąsiedztwo typ [1,2,3] -> [2,1,3],[2,3,1][1,3,2](wymiani kązdy zkażdym)
    return neighbours

In [8]:
def TheBestNeighbour(cities_lenght,city_number,neighbours): #metoda dla najlepszego sąsiada
    best_distance = tourDisatance(cities_lenght,city_number,neighbours[0])
    best_neighbour = neighbours[0]
    for i in neighbours:
        distance = tourDisatance(cities_lenght,city_number,i)
        if distance < best_distance:
            best_distance = distance
            best_neighbour = i
    return best_distance,best_neighbour

In [9]:
def TheBestNeighbour2(cities_lenght,city_number,neighbours,solution_distance): #metoda dla pierwszego lepszego sąsiada. wymiana [1,2,3,4] -> [3,2,1,4] swap wymiana 2 miast miejscami
    best_distance = solution_distance
    best_neighbour = []
    neighbours_method = neighbours.copy()
    while (len(neighbours_method) != 0):
        random_neighbour = random.choice(neighbours_method)
        distance = tourDisatance(cities_lenght,city_number,random_neighbour)
        if distance < best_distance:
            best_distance = distance
            best_neighbour = random_neighbour.copy()
            break
        else:
            neighbours_method.remove(random_neighbour)
    return best_distance,best_neighbour


In [10]:
def IHC(xlsx_data,limit_for_itteration,kind_of_neighbours,max_start_distance):  #wybiera najlepszego sąsiada
    cities_lenght,city_number = df_to_array(xlsx_data)
    random_tour = randomTour_max(cities_lenght,city_number,max_start_distance)
    solution_tour = random_tour.copy()
    tour_distance = tourDisatance(cities_lenght,city_number,random_tour)
    solution_distance = tour_distance.copy()
    neighbours = kind_of_neighbours(random_tour,city_number)
    best_distance,best_neighbour = TheBestNeighbour2(cities_lenght,city_number,neighbours,solution_distance)
    number_of_iterration = 0

    while ((solution_distance > best_distance) and (number_of_iterration < limit_for_itteration) ): #jeśli zaszła zmiana najlepszego wyniku (nie byliśmy w ekstremum.)
        solution_tour = best_neighbour
        solution_distance = best_distance
        neighbours = kind_of_neighbours(solution_tour,city_number)
        best_distance,best_neighbour = TheBestNeighbour2(cities_lenght,city_number,neighbours,solution_distance)
        number_of_iterration += 1 


    return solution_tour,solution_distance,random_tour,tour_distance,number_of_iterration


    

In [11]:

def IHC2(xlsx_data,limit): #wybiera pierwszego lepszego sąsiada
    cities_lenght,city_number = df_to_array(xlsx_data)
    random_tour = randomTour(cities_lenght,city_number)
    solution_tour = random_tour.copy()
    tour_distance = tourDisatance(cities_lenght,city_number,random_tour)
    solution_distance = tour_distance
    neighbours = Neighbours2(random_tour,city_number)
    best_distance,best_neighbour = TheBestNeighbour2(cities_lenght,city_number,neighbours,solution_distance)
    x = 0

    while ((solution_distance > best_distance) and (x<limit)): #jeśli zaszła zmiana najlepszego wyniku (nie byliśmy w ekstremum.)
        solution_tour = best_neighbour
        solution_distance = best_distance
        neighbours = Neighbours2(solution_tour,city_number)
        best_distance,best_neighbour = TheBestNeighbour2(cities_lenght,city_number,neighbours,solution_distance)
        x += 1 


    return solution_tour,solution_distance,random_tour,tour_distance,x

In [20]:
solution_tour,solution_distance,random_tour,tour_disatance,x= IHC(tsp127,1000,Neighbours2,580000)

print(f"""Pierwsza wylosowana trasa:

{random_tour}

długość trasy: {tour_disatance}

Znalezenienie ekstremum za pomocą metody (wspinaczkowej) IHC
Najelpsza trasa: 
{solution_tour}

długość trasy: {solution_distance}

print: liczba iteracji: {x}
 """)

KeyboardInterrupt: 

In [41]:
def TheBestIHC(number_start,xlsx_data,limit,kind_of_neighbours,max_start_distance):
    best_tour,best_result,_,_,iteration_number= IHC(xlsx_data,limit,kind_of_neighbours,max_start_distance)
    for i in range(0,number_start):
        solution_tour,solution_distance,random_tour,tour_distance,number_of_iterration= IHC(xlsx_data,limit,kind_of_neighbours,max_start_distance)
        if (best_result > solution_distance) :
            best_result = solution_distance
            best_tour = solution_tour
            iteration_number = number_of_iterration
            best_tour_show = best_tour.copy()
            best_tour_show = [x+1 for x in best_tour_show]
            print(f"""
            wynik : {best_result} , liczba iteracji wewnątrz algorytmu:{iteration_number}
            trasa: {best_tour_show}
            """) 
    return best_result,best_tour_show,iteration_number

In [34]:
best_result,best_tour,iteration_number = TheBestIHC(100,tsp29,200,Neighbours2,40000)
print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_tour}\n Liczba iteracji w najlepszej próbie : {iteration_number}")



            wynik : 2135,58
            trasa: [1, 21, 13, 19, 15, 18, 17, 14, 22, 11, 4, 10, 20, 2, 3, 29, 26, 5, 9, 12, 6, 28, 8, 27, 23, 7, 25, 16, 24]
            

            wynik : 2113,67
            trasa: [6, 12, 9, 5, 21, 1, 28, 8, 24, 27, 23, 7, 25, 11, 22, 14, 17, 18, 15, 4, 19, 16, 13, 10, 20, 2, 3, 29, 26]
            

            wynik : 2080,67
            trasa: [9, 12, 6, 28, 1, 8, 16, 19, 4, 15, 18, 17, 14, 22, 11, 25, 7, 23, 27, 24, 13, 10, 20, 21, 2, 3, 29, 26, 5]
            

            wynik : 2055,67
            trasa: [26, 29, 3, 2, 21, 1, 8, 27, 23, 7, 25, 19, 11, 22, 14, 17, 18, 15, 4, 10, 20, 13, 16, 24, 28, 6, 12, 9, 5]
            

            wynik : 2028,60
            trasa: [4, 20, 10, 13, 21, 2, 3, 29, 26, 5, 9, 12, 6, 28, 1, 8, 24, 16, 27, 23, 7, 25, 19, 11, 22, 14, 17, 18, 15]
            

            wynik : 2026,60
            trasa: [16, 27, 23, 7, 25, 19, 11, 22, 14, 17, 18, 15, 4, 10, 20, 21, 2, 3, 29, 26, 5, 9, 12, 6, 28, 1, 8, 24, 13]

In [ ]:
start_iteration = [20,100,1000,3000]
ihc_iteration = [20,50,100,200]
kind_neighbours = [Neighbours,Neighbours2]
start_distance = [4800,5200,5800]
data = [tsp29,tsp48,tsp76,tsp127]

In [ ]:

for kn in kind_neighbours:
    for ii in ihc_iteration:
        for sd in  start_distance:
            d20 = 0
            minim = 10000
            for i in range(20):
                best_result,best_tour,iteration_number = TheBestIHC(100,tsp29,ii,kn,sd)
                d20 += best_result
                if (best_result < minim) :
                    minim = best_result
            print(f""" 
                      29
            sąsiedztwo: {kn.__name__}
            liczba iteracji wewnąrtrz algorytmu: {ii}
            start distance: {sd}
            najlepszy wynik: {minim}
            średnia: {d20/20}""")


KeyboardInterrupt: 

In [ ]:
ihc_iteration = [100,150,200,300]
kind_neighbours = [Neighbours,Neighbours2]
data = [tsp29,tsp48,tsp76,tsp127]
for ii in ihc_iteration:
    d20 = 0
    minim = 100000
    for i in range(10):
        best_result,best_tour,iteration_number = TheBestIHC(30,tsp48,ii,kn,100000)
        d20 += best_result
        if (best_result < minim) :
            minim = best_result
    print(f""" 
                      29
            sąsiedztwo: {kn.__name__}
            liczba iteracji wewnąrtrz algorytmu: {ii}
            najlepszy wynik: {minim}
            średnia: {d20/20}""")

 
                      29
            sąsiedztwo: Neighbours
            liczba iteracji wewnąrtrz algorytmu: 100
            najlepszy wynik: 13102
            średnia: 7036.75
 
                      29
            sąsiedztwo: Neighbours2
            liczba iteracji wewnąrtrz algorytmu: 100
            najlepszy wynik: 11973
            średnia: 6415.45
 
                      29
            sąsiedztwo: Neighbours
            liczba iteracji wewnąrtrz algorytmu: 150
            najlepszy wynik: 11456
            średnia: 6284.5
 
                      29
            sąsiedztwo: Neighbours2
            liczba iteracji wewnąrtrz algorytmu: 150
            najlepszy wynik: 10882
            średnia: 5561.95
 
                      29
            sąsiedztwo: Neighbours
            liczba iteracji wewnąrtrz algorytmu: 200
            najlepszy wynik: 11656
            średnia: 6236.55
 
                      29
            sąsiedztwo: Neighbours2
            liczba iteracji wewnąrtrz alg

In [ ]:
start_distance = [45000,40000]
kind_neighbours = [Neighbours,Neighbours2]
data = [tsp29,tsp48,tsp76,tsp127]
# sparawdzanie zależnosći od liczby iteracji przejścia między rozwiązaniami w 1 algorytmiedla miasta 48
for kn in kind_neighbours:
    d20 = 0
    minim = 1000000
    for i in range(15):
        best_result,best_tour,iteration_number = TheBestIHC(10,tsp76,1000,kn,700000)
        d20 += best_result
        if (best_result < minim) :
            minim = best_result
    print(f""" 
                      29
            sąsiedztwo: {kn.__name__}
            start_distance: {sd}
            najlepszy wynik: {minim}
            średnia: {d20/10}""")

 
                      29
            sąsiedztwo: Neighbours
            start_distance: 4800
            najlepszy wynik: 143191.0792494091
            średnia: 228331.16404923834
 
                      29
            sąsiedztwo: Neighbours2
            start_distance: 4800
            najlepszy wynik: 116031.2614662515
            średnia: 180399.86707606987


In [ ]:
start_distance = [700000,500000]
kind_neighbours = [Neighbours,Neighbours2]
data = [tsp29,tsp48,tsp76,tsp127]
# sparawdzanie zależnosći od liczby iteracji przejścia między rozwiązaniami w 1 algorytmiedla miasta 48
for sd in start_distance:
    d20 = 0
    minim = 1000000
    for i in range(10):
        best_result,best_tour,iteration_number = TheBestIHC(10,tsp76,1000,Neighbours2,sd)
        d20 += best_result
        if (best_result < minim) :
            minim = best_result
    print(f""" 
                      29
            start_distance: {sd}
            najlepszy wynik: {minim}
            średnia: {d20/10}""")

 
                      29
            start_distance: 700000
            najlepszy wynik: 118221.8122278118
            średnia: 120395.16389732447
 
                      29
            start_distance: 500000
            najlepszy wynik: 114076.76291057507
            średnia: 118259.41617452819


In [ ]:
start_distance = [700000,500000]
kind_neighbours = [Neighbours,Neighbours2]
ihc_iteration = [250,500,1000,1500]
data = [tsp29,tsp48,tsp76,tsp127]
# sparawdzanie zależnosći od liczby iteracji przejścia między rozwiązaniami w 1 algorytmiedla miasta 48
for ii in ihc_iteration:
    d20 = 0
    minim = 1000000
    for i in range(10):
        best_result,best_tour,iteration_number = TheBestIHC(10,tsp76,ii,Neighbours2,700000)
        d20 += best_result
        if (best_result < minim) :
            minim = best_result
    print(f""" 
                      29
            ihc_itteration: {ii}
            najlepszy wynik: {minim}
            średnia: {d20/10}""")

 
                      29
            ihc_itteration: 250
            najlepszy wynik: 117118.76304480391
            średnia: 122182.64070515498
 
                      29
            ihc_itteration: 500
            najlepszy wynik: 114370.28482569216
            średnia: 119233.52596977758
 
                      29
            ihc_itteration: 1000
            najlepszy wynik: 114065.05573028205
            średnia: 119726.00457174517
 
                      29
            ihc_itteration: 1500
            najlepszy wynik: 116520.36369589258
            średnia: 119750.04454959653


In [ ]:
kind_neighbours = [Neighbours,Neighbours2]
data = [tsp29,tsp48,tsp76,tsp127]
# sparawdzanie zależnosći od liczby iteracji przejścia między rozwiązaniami w 1 algorytmiedla miasta 48
for kn in kind_neighbours:
    d20 = 0
    minim = 100000000
    for i in range(7):
        best_result,best_tour,iteration_number = TheBestIHC(3,tsp127,1000,kn,7000000)
        d20 += best_result
        if (best_result < minim) :
            minim = best_result
    print(f""" 
                      29
            sąsiedztwo: {kn.__name__}
            najlepszy wynik: {minim}
            średnia: {d20/7}""")

 
                      29
            sąsiedztwo: Neighbours
            najlepszy wynik: 164483.2913280919
            średnia: 172007.96805934855
 
                      29
            sąsiedztwo: Neighbours2
            najlepszy wynik: 134526.87246530387
            średnia: 140990.4728570153


In [ ]:
start_distance = [580000,650000,700000]
# sparawdzanie zależnosći od liczby iteracji przejścia między rozwiązaniami w 1 algorytmiedla miasta 48
for sd in start_distance:
    d20 = 0
    minim = 10000000
    for i in range(7):
        best_result,best_tour,iteration_number = TheBestIHC(3,tsp127,1000,Neighbours2,sd)
        d20 += best_result
        if (best_result < minim) :
            minim = best_result
    print(f""" 
                      29
            start_distance: {sd}
            najlepszy wynik: {minim}
            średnia: {d20/7}""")

 
                      29
            start_distance: 580000
            najlepszy wynik: 131052.66212914896
            średnia: 139604.9416642839
 
                      29
            start_distance: 650000
            najlepszy wynik: 136063.46304754433
            średnia: 141227.36141789134
 
                      29
            start_distance: 700000
            najlepszy wynik: 135668.90006046311
            średnia: 140701.36829006733


In [ ]:
start_distance = [45000,40000]
kind_neighbours = [Neighbours,Neighbours2]
data = [tsp29,tsp48,tsp76,tsp127]
ihc_iteration = [500,1000,2000]
# sparawdzanie zależnosći od liczby iteracji przejścia między rozwiązaniami w 1 algorytmiedla miasta 48
for ii in ihc_iteration:
    d20 = 0
    minim = 1000000
    for i in range(7):
        best_result,best_tour,iteration_number = TheBestIHC(10000,tsp127,ii,Neighbours2,700000)
        d20 += best_result
        if (best_result < minim) :
            minim = best_result
    print(f""" 
                      29
            ihc_itteration: {ii}
            najlepszy wynik: {minim}
            średnia: {d20/7}""")

 
                      29
            ihc_itteration: 500
            najlepszy wynik: 139217.82647168092
            średnia: 146387.34765914438
 
                      29
            ihc_itteration: 1000
            najlepszy wynik: 136125.80707916868
            średnia: 140081.3992291924
 
                      29
            ihc_itteration: 2000
            najlepszy wynik: 135558.35183995217
            średnia: 140900.31940624217


In [37]:
best_result,best_tour,iteration_number = TheBestIHC(10000,tsp29,100,Neighbours2,4800)
print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_tour}\n Liczba iteracji w najlepszej próbie : {iteration_number}")


            wynik : 2071 , liczba iteracji wewnątrz algorytmu:51
            trasa: [13, 16, 24, 1, 28, 8, 27, 23, 7, 25, 19, 11, 22, 14, 17, 18, 15, 4, 10, 20, 2, 3, 29, 26, 5, 9, 12, 6, 21]
            

            wynik : 2050 , liczba iteracji wewnątrz algorytmu:57
            trasa: [3, 2, 13, 16, 24, 8, 27, 23, 7, 25, 19, 11, 22, 14, 17, 18, 15, 4, 10, 20, 21, 1, 28, 6, 12, 9, 5, 26, 29]
            

            wynik : 2034 , liczba iteracji wewnątrz algorytmu:40
            trasa: [6, 28, 8, 27, 23, 7, 25, 19, 11, 22, 14, 17, 18, 15, 4, 10, 20, 13, 16, 24, 1, 21, 2, 3, 29, 26, 5, 9, 12]
            

            wynik : 2033 , liczba iteracji wewnątrz algorytmu:56
            trasa: [10, 13, 16, 19, 4, 15, 18, 17, 14, 22, 11, 25, 7, 23, 27, 8, 24, 1, 28, 6, 12, 9, 5, 26, 29, 3, 2, 21, 20]
            

            wynik : 2026 , liczba iteracji wewnątrz algorytmu:50
            trasa: [11, 22, 14, 17, 18, 15, 4, 10, 20, 21, 2, 3, 29, 26, 5, 9, 12, 6, 28, 1, 8, 24, 13, 16, 27

In [38]:
best_result,best_tour,iteration_number = TheBestIHC(4000,tsp48,200,Neighbours2,45000)
print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_tour}\n Liczba iteracji w najlepszej próbie : {iteration_number}")


            wynik : 12191 , liczba iteracji wewnątrz algorytmu:124
            trasa: [21, 47, 20, 36, 30, 43, 17, 27, 19, 37, 6, 28, 7, 18, 44, 31, 38, 9, 8, 1, 22, 16, 41, 34, 29, 2, 4, 26, 42, 48, 25, 13, 11, 12, 33, 46, 15, 40, 3, 23, 14, 5, 10, 35, 45, 24, 32, 39]
            

            wynik : 11313 , liczba iteracji wewnątrz algorytmu:129
            trasa: [7, 18, 44, 31, 38, 46, 33, 20, 47, 21, 39, 32, 24, 10, 45, 35, 4, 26, 42, 2, 29, 5, 48, 25, 14, 13, 23, 40, 9, 8, 1, 22, 16, 41, 34, 3, 11, 12, 15, 36, 6, 37, 19, 27, 17, 43, 30, 28]
            

            wynik : 10880 , liczba iteracji wewnątrz algorytmu:163
            trasa: [30, 43, 17, 27, 19, 37, 6, 28, 7, 18, 44, 31, 38, 9, 8, 1, 3, 23, 14, 25, 13, 11, 12, 15, 40, 22, 16, 41, 34, 29, 2, 26, 4, 35, 45, 10, 24, 42, 5, 48, 39, 32, 21, 47, 20, 33, 46, 36]
            

            wynik : 10807 , liczba iteracji wewnątrz algorytmu:121
            trasa: [14, 34, 3, 22, 16, 41, 29, 2, 26, 4, 35, 45, 10, 24, 42, 5, 

In [39]:
best_result,best_tour,iteration_number = TheBestIHC(1000,tsp76,500,Neighbours2,500000)
print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_tour}\n Liczba iteracji w najlepszej próbie : {iteration_number}")


            wynik : 121761.06267724869 , liczba iteracji wewnątrz algorytmu:274
            trasa: [25, 22, 23, 1, 76, 75, 2, 8, 7, 6, 9, 10, 11, 12, 13, 14, 74, 15, 16, 17, 18, 37, 36, 35, 34, 33, 32, 29, 27, 48, 44, 30, 31, 19, 20, 5, 3, 4, 21, 24, 46, 45, 47, 69, 68, 70, 67, 50, 49, 51, 66, 65, 71, 72, 73, 64, 63, 62, 61, 41, 40, 38, 39, 60, 59, 58, 57, 56, 55, 52, 53, 54, 42, 43, 28, 26]
            

            wynik : 118609.83757904338 , liczba iteracji wewnątrz algorytmu:243
            trasa: [29, 30, 31, 19, 20, 5, 4, 3, 6, 9, 10, 11, 35, 34, 55, 56, 49, 50, 67, 70, 68, 69, 47, 48, 44, 43, 42, 54, 53, 52, 51, 66, 65, 71, 72, 73, 64, 63, 62, 61, 57, 58, 59, 60, 41, 40, 39, 38, 36, 37, 18, 17, 16, 15, 74, 14, 13, 12, 8, 7, 2, 75, 76, 1, 23, 22, 21, 25, 24, 46, 45, 27, 26, 28, 33, 32]
            

            wynik : 117847.59370297268 , liczba iteracji wewnątrz algorytmu:242
            trasa: [52, 53, 54, 42, 43, 28, 29, 30, 31, 19, 20, 5, 3, 2, 75, 76, 1, 23, 22, 24, 46, 4

In [42]:
best_result,best_tour,iteration_number = TheBestIHC(100,tsp127,1000,Neighbours2,580000)
print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_tour}\n Liczba iteracji w najlepszej próbie : {iteration_number}")


            wynik : 142668.00949413198 , liczba iteracji wewnątrz algorytmu:509
            trasa: [12, 18, 77, 76, 78, 117, 84, 81, 126, 83, 101, 102, 82, 75, 70, 71, 110, 85, 86, 87, 88, 109, 96, 119, 63, 69, 68, 74, 73, 67, 8, 72, 19, 22, 20, 14, 41, 36, 35, 54, 48, 118, 53, 56, 50, 115, 13, 105, 15, 108, 17, 21, 4, 23, 24, 6, 106, 114, 7, 2, 51, 124, 52, 5, 121, 57, 45, 103, 44, 40, 34, 39, 42, 43, 37, 16, 1, 120, 10, 100, 58, 64, 113, 65, 99, 92, 89, 125, 104, 59, 60, 62, 61, 91, 90, 116, 9, 11, 3, 66, 55, 47, 49, 46, 94, 112, 111, 107, 127, 93, 95, 123, 28, 29, 32, 98, 97, 122, 33, 25, 80, 79, 31, 27, 26, 38, 30]
            

            wynik : 138178.32826029544 , liczba iteracji wewnątrz algorytmu:492
            trasa: [25, 38, 39, 42, 34, 43, 36, 37, 16, 1, 7, 105, 15, 20, 108, 4, 19, 72, 8, 24, 6, 106, 114, 120, 13, 2, 51, 121, 5, 50, 115, 10, 3, 11, 9, 23, 22, 17, 21, 77, 76, 78, 80, 79, 12, 14, 41, 30, 40, 35, 44, 45, 103, 123, 98, 97, 122, 33, 26, 27, 31, 18, 74, 73, 6